In [38]:
import numpy as np
import pandas as pd
import os 
from PIL import Image
from glob import glob   # takes care of various extension of image files

### Generating Datasets

In [39]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset

from sklearn.utils import shuffle

In [40]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
    print('gpu')
else:
    device=torch.device('cpu')
    print('cpu')

gpu


In [64]:
# Creating child class 'BrainMRIDataset' from parent class 'Dataset'
class BrainMRIDataset(Dataset):

    def __init__(self, data_dir, reshape=True, height=128, width=128, autoencoder=False):
        self.data_dir = data_dir
        # No Brain Tumor MRIDataset
        self.no_class = glob(data_dir+'/no/*')
        # Yes Brain MRIDataset
        self.yes_class = glob(data_dir+'/yes/*')

        self.height = height
        self.width = width
        self.reshape = reshape
        self.autoencoder = autoencoder

        # return 0 if no tumor
        labels = [0 for i in range(len(self.no_class))]
        # return 1 if tumor exists
        labels += [1 for i in range(len(self.yes_class))]

        image_links = self.no_class + self.yes_class

        self.df = pd.DataFrame({'image': image_links, 'labels': labels})
        self.df.reset_index(inplace=True, drop=True)

    def __len__(self):
        return len(self.no_class)+len(self.yes_class)

    def __getitem__(self, idx):
        image_list = self.df['image'][idx]
        label_list = self.df['labels'][idx]

        if type(image_list) == str:
            image_list = [image_list]

        if not isinstance(label_list.values,np.int64):
            label_list = label_list.values

        image_array = []
        
        for image in image_list:
            image = Image.open(image).convert("L")

            if self.reshape:
                image = image.resize((self.height,self.width))

        array = np.asarray(image)
        array = array.reshape(1,self.height,self.width)
        image_array.append(array)

        return [torch.tensor(image_array,device=device),
                torch.tensor(label_list,device=device)]

    def __repr__(self):
        return str(self.df.sample(10))

In [65]:

dataset = BrainMRIDataset("brain_tumor_dataset")

In [66]:
print(dataset)

                                image  labels
204   brain_tumor_dataset/yes\Y42.jpg       1
189   brain_tumor_dataset/yes\Y29.jpg       1
178  brain_tumor_dataset/yes\Y252.jpg       1
125   brain_tumor_dataset/yes\Y15.jpg       1
22   brain_tumor_dataset/no\30 no.jpg       0
155  brain_tumor_dataset/yes\Y188.jpg       1
236   brain_tumor_dataset/yes\Y79.jpg       1
216   brain_tumor_dataset/yes\Y56.jpg       1
109  brain_tumor_dataset/yes\Y109.JPG       1
29   brain_tumor_dataset/no\37 no.jpg       0


### CNN Model


In [102]:
class BrainTumorCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 256, kernel_size=3),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 32, kernel_size=2)
        )

        self.linear1 = nn.Linear(62, 128)
        self.linear2 = nn.Linear(128, 64)
        self.flat = nn.Flatten(1)
        self.linear3 = nn.Linear(126976,2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        x = self.flat(x)
        x = self.linear3(x)

        return x

In [103]:
model = BrainTumorCNN()
model.to(device)

BrainTumorCNN(
  (conv1): Sequential(
    (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(256, 32, kernel_size=(2, 2), stride=(1, 1))
  )
  (linear1): Linear(in_features=62, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linear3): Linear(in_features=126976, out_features=2, bias=True)
)

In [104]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)